<a href="https://colab.research.google.com/github/HumayDS/Big-data-analysis/blob/main/Lesson_10_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [ ]:
##Data cekilir
df = pd.read_csv("https://raw.githubusercontent.com/HumayDS/Big-data-analysis/main/diabetes.csv")
df.drop('Unnamed: 0' , axis = 1, inplace = True)
df = df.dropna()
df.head(50)

In [13]:
# Target deyishken haqqinda melumat alinir
df["Outcome"].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [14]:
#Xler ve y mueyyenlesdirilir
y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

In [19]:
#Aciqlayici statsmodel kitabxanasindan logistic regression modeli qurulur
loj = sm.Logit(y, X)
loj_model= loj.fit()
loj_model.summary()

Optimization terminated successfully.
         Current function value: 0.608498
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Outcome   No. Observations:                  768
Model:                          Logit   Df Residuals:                      760
Method:                           MLE   Df Model:                            7
Date:                Fri, 17 Nov 2023   Pseudo R-squ.:                 0.05922
Time:                        05:21:33   Log-Likelihood:                -467.33
converged:                       True   LL-Null:                       -496.74
Covariance Type:            nonrobust   LLR p-value:                 2.583e-10
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Pregnancies                  0.1284      0.029      4.484      0.000       0.072       0.185
Glucose                      0.0129      0.003      4.757      0.000       0.008       0.018
BloodPressure               -0.0303      0.005     -6.481      0.000      -0.039      -0.021
SkinThickness                0.0002      0.006      0.032      0.974      -0.012       0.012
Insulin                      0.0007      0.001      0.942      0.346      -0.001       0.002
BMI                         -0.0048      0.011     -0.449      0.653      -0.026       0.016
DiabetesPedigreeFunction     0.3203      0.240      1.335      0.182      -0.150       0.790
Age                         -0.0156      0.008     -1.852      0.064      -0.032       0.001
============================================================================================
"""

In [20]:
#Test -traine bolunmemish, umumi datada uzerinde model qurulur
from sklearn.linear_model import LogisticRegression
loj = LogisticRegression(solver = "liblinear")
loj_model = loj.fit(X,y)
loj_model

LogisticRegression(solver='liblinear')

In [21]:
#Intercept ve emsallara baxilir
print(loj_model.intercept_)
print(loj_model.coef_)

[-5.89249766]
[[ 1.17061865e-01  2.83893855e-02 -1.68880903e-02  7.55635388e-04
  -6.42919999e-04  5.97757973e-02  6.77489484e-01  7.24167534e-03]]


In [23]:
#qurulmus model esasinda y-ler proqnoz edilir
y_pred = loj_model.predict(X)
y_pred[0:10]

array([1, 0, 1, 0, 1, 0, 0, 1, 1, 0])

In [24]:
##Proqnoz edilmish y-lerin probability-lerine baxiriq
y_probs = loj_model.predict_proba(X)
y_probs = y_probs[:,1]
y_probs[0:10]

array([0.64961419, 0.08296616, 0.77525048, 0.07859921, 0.83273963,
       0.20100061, 0.11979315, 0.72187436, 0.67941413, 0.07724667])

In [ ]:
#Proqnoz olunmus ve real y-lere baxish
i = np.arange(len(y_pred))
Data = pd.DataFrame({'y_real' : pd.Series(y, index=i),'y_pred' : pd.Series(y_pred, index = i), 'y_probs' : pd.Series(y_probs, index = i)})
Data.head(15)

In [27]:
df["Outcome"].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [28]:
#Confusion matrixe baxaq
##Confusion matrix pythonda by default basga formatda gelir
confusion_matrix(y, y_pred).T

array([[448, 121],
       [ 52, 147]])

In [29]:
##Confusion matrixi anlaya bileceyimiz formata saliriq
cm = confusion_matrix(y, y_pred).T
cm = pd.DataFrame(cm)
a0 = pd.DataFrame(cm[0])
a1 = pd.DataFrame(cm[1])
cm = pd.concat([a1,a0],axis = 1)
lst = [1,0]
cm = cm.loc[lst]
cm

,1,0
1,147,52
0,121,448


In [30]:
data2 = pd.DataFrame(({'1' : ["TP" , "FN"] ,
          '0' : ['FP' , 'TN']}))
data2

,1,0
0,TP,FP
1,FN,TN


#Accuracy deyerini hesablayaq (butun duz proqnoz edilenler / butun proqnoz edilenler  ----Proqnoz etdiklerimizin 77%-i duzdur)
#(TP+TN)/(TP+TN+FP+FN)


In [39]:
y_pred = [1 if i > 0.5 else 0 for i in y_probs]

In [40]:
accuracy_score(y, y_pred)

0.7747395833333334

#Presicion-- Deqiqlik --- Diabet olaraq proqnoz etdiklerimizin nece faizi realda diabetdir ?
# TP/(TP+FP)

In [41]:
print(precision_score(y,y_pred))
print(147/(147+52))

0.7386934673366834
0.7386934673366834


#Recall -  Sensitivity  -- True Pozitive rate --  Realda diabet olanlarin nece faizi diabet olaraq proqnoz edildi?
#TP/(TP+FN)

In [42]:
print(recall_score(y,y_pred))
print(147/(147+121))

0.5485074626865671
0.5485074626865671


# F1 Skoru --  2 * (Presicion * recall)/(Presicion + recall)
#Presicion ve Recallin balanslasdirilmis deyeri

In [43]:
f1_score(y,y_pred)

0.6295503211991434

In [44]:
#Dataseti test ve traine ayiririq
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.30,
                                                    random_state = 42)

In [45]:
##TRain set uzerinden modeli qururuq
loj = LogisticRegression(solver = "liblinear")
loj_model = loj.fit(X_train,y_train)
loj_model

LogisticRegression(solver='liblinear')

In [46]:
##Test set uzerinden deyerlendirilir
accuracy_score(y_test, loj_model.predict(X_test))

0.7532467532467533